# Stock Portfolio 
## Owner: Ryne F. Shelton
### Email:rfshelton@att.net - (206) 651-9090
------------------------------------------------------------------
### Filename: rshelton_stockportfolio_example.ipynb
### Description:
Utilizes Machine Learning (ML) libraries to suggest future portfolio performance and inform trade options. 


### Libraries and Packages
#### ML
SciKit-Learn, Tensorflow
#### Data
yFinance

In [3]:
#https://www.analyticsvidhya.com/blog/2021/10/machine-learning-for-stock-market-prediction-with-step-by-step-implementation/
#Importing the Libraries
import pandas as pd
import numpy as np
import yfinance as yf
%matplotlib inline
import pandas_datareader as data
import matplotlib. pyplot as plt
import matplotlib
from sklearn.preprocessing import MinMaxScaler
from keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib. dates as mandates
from sklearn.preprocessing import MinMaxScaler
from sklearn import linear_model
from keras.models import Sequential
from keras.layers import Dense
import keras.backend as K
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.models import load_model
from keras.layers import LSTM
from keras.utils import plot_model

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2024-03-30 14:06:33.936274: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-30 14:06:33.940006: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-30 14:06:33.999494: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-30 14:06:35.132188: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


#### Global Variables 
Start/End Analysis Dates, Stock Ticker Symbol

In [49]:
# Date format: 'yyyy-mm-dd'
start_date = '2018-03-23'
end_date   = '2023-03-23'
# Stock Tickers
ticker_dict = {
    'ARKK':0.49,
    'VWO':0.28,
    'SPMO':0.15,
    'VEA':0.32,
    'VONG':0.16,
    'IVV':0.08,
    'QUAL':0.06,
    'BND':0.19,
    'VB':.04,
    'BX':0.08,
    'PCRFY':5,
    'TSLA':0.04,
    'AMD':0.28,
    'INTC':1.26
}

In [14]:
ticker_keys = list(ticker_dict.keys())

In [16]:
len(ticker_keys)

14

In [95]:
class rfs_stocks:
    # saves passed in dictionary 
    def __init__(self, ticker_dict,start_date, end_date):
        self.tickers = ticker_dict
        self.ticker_keys = list(self.tickers.keys())
        self.start_date = start_date
        self.end_date = end_date        
    # gets ticker daily price and volume details
    def portfolio_details(self):
        portfolio_total = 0
        print('Fetching info for ',len(self.ticker_keys),'stocks.')
        print('********************************************')
        for i in range(len(self.ticker_keys)):
            ticker_value = self.ticker_keys[i]
            ticker = yf.Ticker(ticker_value).info
            marketPrice = ticker['regularMarketOpen']
            previousClosePrice = ticker['regularMarketPreviousClose']
            ticker_holdings = marketPrice*self.tickers[self.ticker_keys[i]]
            print('Ticker Value: '+ticker_value)
            print('--------------------------------------------')
            print('Market Price Value:', marketPrice)
            print('Previous Close Price Value:', previousClosePrice)
            print('Holdings Value: ',ticker_holdings)
            portfolio_total = portfolio_total+ticker_holdings
            print('============================================')
        print('Portfolio total worth: ', portfolio_total)
    # retrieves stock data from tickers specified in stored ticker dictionary  
    # retrieved data is stored in a dict called 'data' as dataframes
    def get_ticker_data(self,db=False):
        self.data={}
        for i in range(len(self.ticker_keys)):
            if db == True:
                print(self.ticker_keys[i])
            self.data[self.ticker_keys[i]] = yf.download(self.ticker_keys[i],self.start_date,self.end_date).add_prefix(self.ticker_keys[i]+' ')
            if db == True:
                print(self.data[self.ticker_keys[i]])
                
    # Combines data contained in the dict of dataframes into a single dataframe
    def unify_data(self,db=False):
        self.get_ticker_data()
        self.unified_data = self.data[self.ticker_keys[0]]
        for i in range(len(self.ticker_keys)):
            i =++i
            self.unified_data = pd.merge(self.unified_data,self.data[self.ticker_keys[i]],on='Date')
        self.unified_data.bfill(inplace=True)
        
                
            
        
        

In [96]:
p1 = rfs_stocks(ticker_dict,start_date,end_date)


In [97]:
p1.portfolio_details()

Fetching info for  14 stocks.
********************************************
Ticker Value: ARKK
--------------------------------------------
Market Price Value: 50.22
Previous Close Price Value: 50.11
Holdings Value:  24.607799999999997
Ticker Value: VWO
--------------------------------------------
Market Price Value: 41.71
Previous Close Price Value: 41.63
Holdings Value:  11.6788
Ticker Value: SPMO
--------------------------------------------
Market Price Value: 80.15
Previous Close Price Value: 80.3
Holdings Value:  12.0225
Ticker Value: VEA
--------------------------------------------
Market Price Value: 50.1
Previous Close Price Value: 50.27
Holdings Value:  16.032
Ticker Value: VONG
--------------------------------------------
Market Price Value: 86.89
Previous Close Price Value: 86.88
Holdings Value:  13.9024
Ticker Value: IVV
--------------------------------------------
Market Price Value: 525.84
Previous Close Price Value: 525.72
Holdings Value:  42.06720000000001
Ticker Value: 

In [98]:
yf.download(ticker_keys[4],start_date,end_date)

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-03-23,35.142502,35.189999,34.334999,34.369999,32.554367,334800
2018-03-26,34.900002,35.380001,34.555000,35.349998,33.482594,184400
2018-03-27,35.527500,35.527500,34.312500,34.557499,32.731960,268000
2018-03-28,34.459999,34.689999,34.189999,34.355000,32.540157,539200
2018-03-29,34.512501,35.154999,34.362499,34.922501,33.077671,451600
...,...,...,...,...,...,...
2023-03-16,58.840000,60.470001,58.730000,60.439999,59.855885,657100
2023-03-17,60.389999,60.700001,59.700001,60.049999,59.469654,696200
2023-03-20,60.049999,60.380001,59.730000,60.360001,59.776661,1278400


In [99]:
p1.unify_data()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [100]:
p1.unified_data

,ARKK Open_x,ARKK High_x,ARKK Low_x,ARKK Close_x,ARKK Adj Close_x,ARKK Volume_x,ARKK Open_y,ARKK High_y,ARKK Low_y,ARKK Close_y,...,AMD Low,AMD Close,AMD Adj Close,AMD Volume,INTC Open,INTC High,INTC Low,INTC Close,INTC Adj Close,INTC Volume
Date,,,,,,,,,,,,,,,,,,,,,
2018-03-23,41.400002,41.509998,40.570999,40.650002,37.988304,420100,41.400002,41.509998,40.570999,40.650002,...,10.570000,10.630000,10.630000,54844500,50.590000,50.990002,49.360001,49.360001,42.153816,37683900
2018-03-26,41.410000,41.720001,40.380001,41.509998,38.791985,312000,41.410000,41.720001,40.380001,41.509998,...,10.180000,10.440000,10.440000,75878500,50.740002,52.490002,50.320000,52.480000,44.818329,39306200
2018-03-27,41.860001,41.900002,39.200001,39.389999,36.810806,615400,41.860001,41.900002,39.200001,39.389999,...,9.900000,10.000000,10.000000,68551300,53.009998,53.250000,50.779999,51.189999,43.716652,38777000
2018-03-28,39.080002,39.259998,37.930000,38.389999,35.876286,654300,39.080002,39.259998,37.930000,38.389999,...,9.790000,9.810000,9.810000,64832100,50.500000,50.700001,49.009998,49.599998,42.358776,37970900
2018-03-29,38.689999,39.387001,38.139999,39.070000,36.511761,352200,38.689999,39.387001,38.139999,39.070000,...,9.810000,10.050000,10.050000,48681400,49.980000,52.080002,49.869999,52.080002,44.476711,45265800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-16,38.529999,39.485001,37.919998,39.240002,38.915455,27952200,38.529999,39.485001,37.919998,39.240002,...,89.019997,96.599998,96.599998,115839200,28.389999,30.270000,28.389999,30.180000,29.763771,75779500
2023-03-17,39.049999,39.369999,38.090000,38.680000,38.360085,28008200,39.049999,39.369999,38.090000,38.680000,...,95.940002,97.839996,97.839996,94080800,29.920000,31.000000,29.440001,29.809999,29.398872,80628500
2023-03-20,38.610001,38.840000,37.645000,38.220001,37.903889,21337400,38.610001,38.840000,37.645000,38.220001,...,92.900002,96.809998,96.809998,92008900,29.799999,29.900000,28.629999,29.160000,28.757839,38780100
